In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pickle
import time
import os
import json
import seaborn as sns
import tensorflow as tf
import random
import cv2
import copy

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

from tensorflow.keras.applications import VGG16,InceptionV3
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential

from keras.applications import VGG16
from keras.applications.resnet import ResNet152
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

from pandas import json_normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score,classification_report
from sklearn.preprocessing import normalize,StandardScaler,RobustScaler,MinMaxScaler
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist, cifar10

from tensorflow import keras
from keras.utils import to_categorical
from keras import regularizers
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D
from keras.losses import binary_crossentropy

from scipy.stats import skew
from scipy.optimize import linear_sum_assignment

from tqdm import tqdm
from PIL import Image
from sklearn import metrics

from scipy.optimize import linear_sum_assignment as linear_assignment
from deep_sort.deep_sort import tracker as Tracker
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort import nn_matching
from deep_sort.tools import generate_detections as gdet

In [ ]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

# tf.compat.v1.disable_eager_execution()
# sess = tf.compat.v1.Session()

### Loading Trained InceptionV3 Model

In [2]:
model = tf.keras.models.load_model('models/Inception_v3_custom/')

Metal device set to: Apple M1 Max


2023-05-17 08:41:48.800878: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-17 08:41:48.801020: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### Loading Yolo

In [161]:
object_detection_model = cv2.dnn.readNet("models/yolov4-tiny.weights", "models/yolov4-tiny.cfg")

### Reading in Video

In [3]:
video_path = 'Datasets/Video/QueenAnnRoy/moving/test.mov'
output_dir = 'Datasets/Video/QueenAnnRoy/stills_test/'
os.makedirs(output_dir, exist_ok=True)
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [5]:
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [6]:
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
input_size = (416, 416)
input_scale = 1/255.0
confidence_thresh = .1
overlap = .1

In [7]:
classes[:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

In [8]:
cap

<VideoCapture 0x2deecf470>

### Defining Region of Interest (ROI)

In [9]:
roi_coordinates = [(352, 820), (855, 820), (1282, 270),(1082, 270)]

In [10]:
# Create a binary mask image with the same size as the original image
image = cv2.imread('Datasets/Video/QueenAnnRoy/stills_test/frame_000000.jpg')  # Load your image
mask = np.zeros_like(image)

# Convert the coords to numpy
roi_pts = np.array(roi_coordinates, dtype=np.int32)
cv2.fillPoly(mask, [roi_pts], (255, 255, 255))  # Set ROI pixels to white (255)

roi = cv2.bitwise_and(image, mask)

# cv2.imshow('ROI', roi)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### Initializing DeepSort

In [195]:
def Object_tracking(object_detection_model, video_path, output_path=output_dir, input_size=416, show=False, CLASSES=classes, score_threshold=confidence_thresh, iou_threshold=overlap, rectangle_colors='', Track_only=['cars', 'trucks']):
    # Definition of the parameters
    max_cosine_distance = 0.5
    nn_budget = None

    # Initializing tracker with Subaru CNN
    model_filename = "/Users/ryan/GA/Portfolio/Subaru-Seattle/models/saved_model.pb"
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    times = []

    if video_path:
        vid = cv2.VideoCapture(video_path)  # Detect on video
    else:
        vid = cv2.VideoCapture(0)  # Detect from webcam

    # By default, VideoCapture returns float instead of int
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    codec = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, codec, fps, (width, height))  # Output_path must be .mp4

    NUM_CLASS = read_class_names(CLASSES)
    key_list = list(NUM_CLASS.keys())
    val_list = list(NUM_CLASS.values())
    while True:
        _, img = vid.read()

        try:
            original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        except:
            break
        image_data = image_preprocess(np.copy(original_image), [input_size, input_size])
        image_data = tf.expand_dims(image_data, 0)

        t1 = time.time()
        pred_bbox = object_detection_model.predict(image_data)
        t2 = time.time()

        times.append(t2 - t1)
        times = times[-20:]

        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
        pred_bbox = tf.concat(pred_bbox, axis=0)

        bboxes = postprocess_boxes(pred_bbox, original_image, input_size, score_threshold)
        bboxes = nms(bboxes, iou_threshold, method='nms')

        # Extract bboxes to boxes (x, y, width, height), scores and names
        boxes, scores, names = [], [], []
        for bbox in bboxes:
            if len(Track_only) != 0 and NUM_CLASS[int(bbox[5])] in Track_only or len(Track_only) == 0:
                boxes.append([bbox[0].astype(int), bbox[1].astype(int), bbox[2].astype(int) - bbox[0].astype(int),
                              bbox[3].astype(int) - bbox[1].astype(int)])
                scores.append(bbox[4])
                names.append(NUM_CLASS[int(bbox[5])])

        # Obtain all the detections for the given frame
        boxes = np.array(boxes)
        names = np.array(names)
        scores = np.array(scores)
        features = np.array(encoder(original_image, boxes))
        detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in
                      zip(boxes, scores, names, features)]

        # Pass detections to the deepsort object and obtain the track information
        tracker.predict()
        tracker.update(detections)

        # Obtain info from the tracks
        tracked_bboxes = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()  # Get the corrected/predicted bounding box
            class_name = track.get_class()  # Get the class name of the particular object
            tracking_id = track.track_id  # Get the ID for the particular track
            index = key_list[val_list.index(class_name)]  # Get the predicted object index by object name
            tracked_bboxes.append(bbox.tolist() + [tracking_id, index])  # Structure data that we could use with our draw_bbox function

        ms = sum(times) / len(times) * 1000
        fps = 1000 / ms

        # Draw detection on frame
        image = draw_bbox(original_image, tracked_bboxes, CLASSES=CLASSES, tracking=True)
        image = cv2.putText(image, "Time: {:.1f} FPS".format(fps), (0, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,
                            (0, 0, 255), 2)

        if output_path != '':
            out.write(image)
        if show:
            cv2.imshow('output', image)
            if cv2.waitKey(25) & 0xFF == ord("q"):
                cv2.destroyAllWindows()
                break

    cv2.destroyAllWindows()

In [196]:
Object_tracking(object_detection_model,
                    video_path,
                    output_path=output_dir,
                    input_size=416,
                    show=False,
                    CLASSES=classes,
                    score_threshold=confidence_thresh,
                    iou_threshold=overlap,
                    rectangle_colors='',
                    Track_only=['cars', 'trucks'])

OSError: SavedModel file does not exist at: /Users/ryan/GA/Portfolio/Subaru-Seattle/models/saved_model.pb/{saved_model.pbtxt|saved_model.pb}

### Object Detection & Model Application

In [ ]:
with tqdm(total=frame_count, unit='frame') as pbar:
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Apply the ROI mask to the frame
        roi_frame = cv2.bitwise_and(frame, mask)

        # Perform object detection within the ROI
        blob = cv2.dnn.blobFromImage(roi_frame, input_scale, input_size, swapRB=True, crop=False)
        object_detection_model.setInput(blob)
        layer_outputs = object_detection_model.forward(object_detection_model.getUnconnectedOutLayersNames())

        # Process the bounding boxes
        boxes = []
        confidences = []
        class_ids = []
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                class_name = classes[class_id]
                if class_name == 'car' and confidence > confidence_thresh:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    width = int(detection[2] * frame.shape[1])
                    height = int(detection[3] * frame.shape[0])
                    x = int(center_x - width / 2)
                    y = int(center_y - height / 2)
                    boxes.append([x, y, width, height])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Apply non-maximum suppression
        indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, overlap)

        # Extract the detected object features for tracking
        features = np.array(boxes)  # Use bounding boxes as features (replace this with your feature extraction logic)

        # Create Detection instances
        detections = []
        for bbox, confidence, class_id in zip(boxes, confidences, class_ids):
            detection = Detection(bbox, confidence, class_id)
            detections.append(detection)

        # Perform object tracking
        outputs = tracker.update(features)

        for output, bbox in zip(outputs, boxes):
            track_id = output[4]

            # Draw bounding box and track ID on the frame
            x, y, w, h = bbox
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, str(track_id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the processed frame
        cv2.imshow('Video', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        pbar.update(1)

cap.release()
cv2.destroyAllWindows()